In [94]:
# Load in PyTorch's pretrained network
import torchvision.models as models
import torch
import numpy as np
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.nn.functional as func
import pandas as pd
from torch.utils.data.dataset import Dataset
from PIL import Image
from torch.utils.data.sampler import SubsetRandomSampler
from random import shuffle
import matplotlib.pyplot as plt
import os
import sklearn.metrics as sk_metrics

In [95]:
# Plot training losses
def plot_losses(losses):
    plt.scatter(range(len(losses)), losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Losses')
    plt.show()
    
# Plot 10 random images given a data loader with their ground truth values
# Modify function to just plot an image when training and testing?
def plot_images(dataloader):
    classes = ['Cardiomegaly', 'Emphysema', 'Effusion', 'No Finding', 'Hernia',
       'Infiltration', 'Mass', 'Nodule', 'Atelectasis', 'Pneumothorax',
       'Pleural_Thickening', 'Pneumonia', 'Fibrosis', 'Edema',
       'Consolidation']
    it = iter(dataloader)
    for i in range(10):
        idx = np.random.randint(0, 99, 1)[0]
        image, label = [x[idx] for x in next(it)]
        plt.figure(num=None, figsize=(8, 6))
        plt.imshow(image)
        correct_labels = [classes[idx] for (idx, val) in enumerate(vals) if val == 1]
        correct_labels = ", ".join(correct_labels)
        print(f'Correct Labels: {correct_labels}')
        plt.show()

In [96]:
# Add separate transformations for testing and training 
# Augmentatiosn for data?
class CHXData(Dataset):
    def __init__(self, img_dir, path_to_labels, is_trainset):
        self.img_dir = img_dir
        self.data_df = pd.read_csv(path_to_labels)
        self.img_names = self.data_df['Image Index']
        self.labels = np.asarray(self.data_df.loc[:, self.data_df.columns != 'Image Index'])
        
        # Normalization and data augmentation
        self.train_transforms = transforms.Compose([
            transforms.Resize(512),
            # Should the normalization be from the statistics of the training set or the entire set?
            # Normalize by using images specific to this domain,
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(180),
            transforms.RandomCrop(384),
            transforms.ToTensor(),
            transforms.Normalize((0.50546, 0.50546, 0.50546), (0.2319, 0.2319, 0.2319))
        ])
        self.test_transforms = transforms.Compose([
            transforms.Resize(512),
            transforms.ToTensor(),
            transforms.Normalize((0.50546, 0.50546, 0.50546), (0.2319, 0.2319, 0.2319))
        ])
        self.is_train = is_trainset
        
    # Return size of the dataset
    def __len__(self):
        return len(self.data_df)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        # Converts to 3-Channel as Resnet takes in 3 channels
        # seems to repeat the same values for every channel
        if self.is_train:
            image = self.train_transforms(Image.open(img_path).convert('RGB'))
        else:
            image = self.test_transforms(Image.open(img_path).convert('RGB'))
        label = self.labels[idx]
        return (image, label)

In [127]:
class CHXModel():
    def __init__(self):
        self.model = None
        self.losses = None
        self.optimizer = None
        self.model_losses = None
    
    # Freeze or un-freeze model layers as required
    def update_grad(self, grad_val):
        for param in self.model.parameters():
            param.requires_grad = grad_val
            
    # Initial model setup
    def set_up_model(self, n_classes, lr):
        self.model = models.resnet34(pretrained=True, progress=True)
        # Freeze all layers
        self.update_grad(False)
        # Resnet has one fully connected layer, which outputs dimensions of n_classes
        num_ftrs = self.model.fc.in_features
        self.model.fc = torch.nn.Linear(num_ftrs, n_classes)
        self.model.cuda()
        # Use a binary cross-entropy loss function; Applies sigmoid internally (generating probabilities)
        # On the probabilities, cross-entropy loss is computed
        # Because we are doing multilabel, this is better as the value outtputted (unlike softmax)
        # is independent of the other values (while in softmax, probabilities must add up to one)
        self.losses =  torch.nn.BCEWithLogitsLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
    
    def load_checkpoint(self, file_path):
        checkpoint = torch.load(file_path)
        self.model.load_state_dict(checkpoint['state'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        return checkpoint['epoch']

    def save_checkpoint(self, state, filename):
        torch.save(state, filename)
    
    # F1 score? Something else that isn't accuracy as we have imbalanced data and because this is medicine
    def evaluate(self, testloader):
        with torch.no_grad():
            # Go through each batch in the testloader
            for X, y in testloader:
                input_img = X.cuda(non_blocking=True)
                labels = y.float().cuda(non_blocking=True)
                self.optimizer.zero_grad()
                output = self.model(input_img)
                sig = torch.nn.Sigmoid()
                probabilities = sig(output)
                # Check if predictions are greater than 0.5
                predictions = probabilities >= 0.75
                # Implement F1 score properly, batch wise, something else?
                
    
    def train(self, epochs, trainloader, val_loader, checkpoint_path=None):
        # Empty cache before training
        torch.cuda.empty_cache()
        if checkpoint_path != None:
            start_epoch = self.load_checkpoint(checkpoint_path)
        else:
            start_epoch = 0
        for e in range(start_epoch, epochs):
            # Get loss per epoch
            running_loss = 0
            for i, (X, y) in enumerate(trainloader):
                input_img = X.cuda(non_blocking=True)
                labels = y.float().cuda(non_blocking=True)
                self.optimizer.zero_grad()
                output = self.model(input_img)
                loss = self.losses(output, labels)
                loss.backward()
                self.optimizer.step()
                running_loss = running_loss + loss.item()
            # Find the loss for the current epoch
            loss = running_loss/len(trainloader)
            print(f"Epoch {e + 1} - Loss: {loss}")
            # Save model losses so far so we can plot them if we stop half way?
            state = {
                'epoch': e + 1,
                'state': self.model.state_dict(),
                'optimizer': self.optimizer.state_dict()
            }
            checkpoint_path = os.path.join('./checkpoints/checkpoint_epoch_' + str(e + 1) + '.pth.tar') 
            self.save_checkpoint(state, checkpoint_path)
            #self.evaluate(val_loader)

In [128]:
n_classes = 15
learning_rate = 1e-4
chx_model = CHXModel()
chx_model.set_up_model(n_classes, learning_rate)

In [129]:
path_to_train_data = './data/Train/'
path_to_test_data = './data/Test/'
path_to_val_data = './data/Val'
path_to_test_labels = './data/Labels/Test_Labels.csv'
path_to_train_labels = './data/Labels/Train_Labels.csv'
path_to_val_labels='./data/Labels/Val_Labels.csv'

In [134]:
# Create dataset and data loader
test_dataset = CHXData(path_to_test_data, path_to_test_labels, False)
train_dataset = CHXData(path_to_train_data, path_to_train_labels, True)
val_dataset =  CHXData(path_to_val_data, path_to_val_labels, False)

# More num_workers consumes more memory for good for speeding up I/O
# pin_memory=True enables fast data transfer to GPUs
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, pin_memory=True,)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True, pin_memory=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=100, shuffle=True, pin_memory=True)

In [ ]:
# Train with the layers frozen for 5 epochs
# Very slow still? Non_blocking, clearing cache, pin memory?
chx_model.train(5, train_loader, val_loader)

Epoch 1 - Loss: 0.2155927475900491
Epoch 2 - Loss: 0.1934442875933127
Epoch 3 - Loss: 0.1909137600231844


In [8]:
# Everything from the dataloader will be augmented even if im just plotting random exmaples?

In [ ]:
# NEED TO DEAL WITH IMBALANCE DATA
# Update evaluate function to give accuracy

In [135]:
chx_model.train(5, train_loader, val_loader)

Epoch 1 - Loss: 0.19903892925217154
Epoch 2 - Loss: 0.19446507664409315
Epoch 3 - Loss: 0.1921391779214947
Epoch 4 - Loss: 0.1906038391689587
Epoch 5 - Loss: 0.1893047764610418
